In [1]:
#Imports
import pandas as pd
import os
import tensorflow as tf
from sklearn.metrics import mean_absolute_error, mean_squared_error

import sys  
sys.path.append("../../")  
from utils.models import *
from utils.datahandling import *
from utils.modelrunner import *

import wandb
import logging
logging.getLogger("wandb").setLevel(logging.ERROR)
logging.getLogger('tensorflow').setLevel(logging.ERROR)

os.environ['WANDB_SILENT'] = 'true'
os.environ['WANDB_CONSOLE'] = 'off'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Prepare data for users 31-60

In [2]:
sequence_length = 25
batch_size = 16

In [3]:
#Get data 
num_users = 30
start_user_idx = 31

cwd = os.path.normpath(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))))
df = pd.read_csv(cwd+'/data/3final_data/Final_Grossload_dataset.csv', index_col='Date')
df.index = pd.to_datetime(df.index)
df.fillna(0, inplace=True)

# Get the first date from the index
start_date = df.index.min()
# Calculate the end date as one year from the start date
end_date = start_date + pd.DateOffset(years=1)
# Filter the dataframe to only include the first year of data
df = df[(df.index >= start_date) & (df.index < end_date)]

df_array = []
for idx in range(start_user_idx, start_user_idx + num_users):
    df_array.append(df[[f'User{idx}', 'temp', 'rhum', 'wspd', 'PC1', 'hour sin', 'hour cos', f'User{idx}_lag_24hrs']])

#Train, Validation and Test datasets
#X_train, y_train, X_val, y_val, X_test, y_test = {}, {}, {}, {}, {}, {}
X_test, y_test = {}, {}

#Create Train, Validation and Test datasets
for idx, df in enumerate(df_array):
    n = len(df)
    test_df = df[0:]

    # Min max sclaing
    test_df = min_max_scaling(test_df)

    # Sequencing
    test_sequences = create_sequences(test_df, sequence_length)

    #Split into feature and label
    X_test[f'user{idx+31}'], y_test[f'user{idx+31}'] = prepare_data(test_sequences, batch_size)

In [4]:
#Hyperparameters
num_features = df_array[0].shape[1]
horizon = 1
max_epochs = 100

loss = tf.keras.losses.MeanSquaredError()
metrics=[
    tf.keras.metrics.RootMeanSquaredError(), 
    tf.keras.metrics.MeanAbsolutePercentageError(),
    tf.keras.metrics.MeanAbsoluteError(),
]

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,mode='min')
timing_callback = TimingCallback()
custom_callback = CustomCallback()

callbacks=[early_stopping, timing_callback, custom_callback]

def load_and_compile_model(model_path, model_type, custom_objects=None):
    model = tf.keras.models.load_model(model_path, custom_objects=custom_objects, compile=False)
    model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=metrics)
    return model

In [5]:
y = np.loadtxt(f'../../../data/3final_data/Clusters_KMeans10_dtw.csv', delimiter=',').astype(int)
num_clusters = 10
cluster_users = {i: [] for i in range(num_clusters)}

# Iterate through each cluster
for cluster_number in range(num_clusters):
    users_in_cluster = np.where(y == cluster_number)[0] +1
    cluster_users[cluster_number] = users_in_cluster
cluster_users

{0: array([ 7, 14, 18, 22, 23, 25, 29], dtype=int64),
 1: array([6], dtype=int64),
 2: array([ 3,  4,  9, 13, 15, 19, 20, 30], dtype=int64),
 3: array([1], dtype=int64),
 4: array([21], dtype=int64),
 5: array([ 2, 28], dtype=int64),
 6: array([ 5, 10, 11, 12, 24, 26, 27], dtype=int64),
 7: array([8], dtype=int64),
 8: array([17], dtype=int64),
 9: array([16], dtype=int64)}

# Totalload

In [6]:
# Initialize DataFrame to store results
columns = ['User', 'Round']
model_names = ['BiLSTM', 'CNN', 'Transformer', 'soft_dense', 'soft_bilstm', 'topk_dense', 'topk_bilstm']

for name in model_names:
    columns.extend([f'RMSE_LL_{name}', f'MAE_LL_{name}', f'MSE_LL_{name}'])  # Local Learning Metrics
    columns.extend([f'RMSE_FL_{name}', f'MAE_FL_{name}', f'MSE_FL_{name}'])  # Federated Learning Metrics

results_df_totalload = pd.DataFrame(columns=columns)

data = "Totalload"
user=1
cluster=4
#Load models
custom_objects = {'EinsumLayer': EinsumLayer,'TopKLayer': TopKLayer,'ImportanceRegularizationLayer': ImportanceRegularizationLayer}
local_model_paths = {
            'LL_BiLSTM': f'{cwd}/src/time_series/Locallearning/TS_LL_{data}_Forecasting_Benchmark/wandb/TS_LL_{data}_bilstm_u{user}_rd{1}.keras',
            'LL_CNN': f'{cwd}/src/time_series/Locallearning/TS_LL_{data}_Forecasting_Benchmark/wandb/TS_LL_{data}_cnn_u{user}_rd{1}.keras',
            'LL_Transformer': f'{cwd}/src/time_series/Locallearning/TS_LL_{data}_Forecasting_Benchmark/wandb/TS_LL_{data}_transformer_u{user}_rd{1}.keras',
            'LL_soft_bilstm': f'{cwd}/src/time_series/Locallearning/TS_LL_{data}_Forecasting_Moe/wandb/TS_LL_{data}_lstm_soft_moe_u{user}_rd{1}.keras',
            'LL_soft_dense': f'{cwd}/src/time_series/Locallearning/TS_LL_{data}_Forecasting_Moe/wandb/TS_LL_{data}_dense_soft_moe_u{user}_rd{1}.keras',
            'LL_topk_bilstm': f'{cwd}/src/time_series/Locallearning/TS_LL_{data}_Forecasting_Moe/wandb/TS_LL_{data}_lstm_topk_moe_u{user}_rd{1}.keras',
            'LL_topk_dense': f'{cwd}/src/time_series/Locallearning/TS_LL_{data}_Forecasting_Moe/wandb/TS_LL_{data}_dense_topk_moe_u{user}_rd{1}.keras'
        }

#Federated learning
federated_model_paths = {
            'FL_BiLSTM': f'{cwd}/src/time_series/Federatedlearning/TS_FL_{data}_Forecasting_Benchmark/wandb/TS_FL_{data}_global_bilstm_c{cluster}_FLround3.keras',
            'FL_CNN': f'{cwd}/src/time_series/Federatedlearning/TS_FL_{data}_Forecasting_Benchmark/wandb/TS_FL_{data}_global_cnn_c{cluster}_FLround3.keras',
            'FL_Transformer': f'{cwd}/src/time_series/Federatedlearning/TS_FL_{data}_Forecasting_Benchmark/wandb/TS_FL_{data}_global_transformer_c{cluster}_FLround3.keras',
            'FL_soft_bilstm': f'{cwd}/src/time_series/Federatedlearning/TS_FL_{data}_Forecasting_Moe/wandb/TS_FL_{data}_global_soft_bilstm_c{cluster}_FLround3.keras',
            'FL_soft_dense': f'{cwd}/src/time_series/Federatedlearning/TS_FL_{data}_Forecasting_Moe/wandb/TS_FL_{data}_global_soft_dense_c{cluster}_FLround3.keras',
            'FL_topk_bilstm': f'{cwd}/src/time_series/Federatedlearning/TS_FL_{data}_Forecasting_Moe/wandb/TS_FL_{data}_global_topk_bilstm_c{cluster}_FLround3.keras',
            'FL_topk_dense': f'{cwd}/src/time_series/Federatedlearning/TS_FL_{data}_Forecasting_Moe/wandb/TS_FL_{data}_global_topk_dense_c{cluster}_FLround3.keras'
        }

local_models = {name: load_and_compile_model(path, name, custom_objects) for name, path in local_model_paths.items()}
federated_models = {name: load_and_compile_model(path, name, custom_objects) for name, path in federated_model_paths.items()}

In [7]:
# Function to evaluate models
def evaluate_models(models, user_idx, X_test_user, y_test_user):
    user_metrics = {}
    for model_name, model in models.items():
        test_loss, test_rmse, test_mape, test_mae = model.evaluate(X_test_user, y_test_user, batch_size=batch_size, verbose=0)
        user_metrics[f'RMSE_{model_name}'] = test_rmse
        user_metrics[f'MAE_{model_name}'] = test_mae
        user_metrics[f'MSE_{model_name}'] = test_loss
    return user_metrics

# Iterate over each user
for user_idx in range(31, 60):
    
    # Get the test data for the current user
    X_test_user = X_test[f'user{user_idx}']
    y_test_user = y_test[f'user{user_idx}']
    
    # Evaluate local models
    local_metrics = evaluate_models(local_models, user_idx, X_test_user, y_test_user)
    
    # Evaluate federated models
    federated_metrics = evaluate_models(federated_models, user_idx, X_test_user, y_test_user)
    
    # Combine metrics and create a new row
    new_row = {'User': user_idx, 'Round': 1, **local_metrics, **federated_metrics}

    # Add the new row to the DataFrame
    results_df_totalload.loc[len(results_df_totalload)] = new_row

print(results_df_totalload["RMSE_LL_BiLSTM"].mean())
print(results_df_totalload["RMSE_FL_soft_bilstm"].mean())

0.104589336391153
0.09406853377305228


In [16]:
print("Benchmark Mean: ")
print("LSTM: ", results_df_totalload["RMSE_LL_BiLSTM"].mean().round(4), " , Transformer: ", results_df_totalload["RMSE_LL_Transformer"].mean().round(4), " , CNN: ", results_df_totalload["RMSE_LL_CNN"].mean().round(4))
print("MoE: ")
print("Soft Dense: ", results_df_totalload["RMSE_FL_soft_dense"].mean().round(4), "Topk Dense: ", results_df_totalload["RMSE_FL_topk_dense"].mean().round(4))
print("Soft LSTM: ", results_df_totalload["RMSE_FL_soft_bilstm"].mean().round(4), "Topk LSTM: ", results_df_totalload["RMSE_FL_topk_bilstm"].mean().round(4))

Benchmark Mean: 
LSTM:  0.1046  , Transformer:  0.1369  , CNN:  0.1383
MoE: 
Soft Dense:  0.0994 Topk Dense:  0.097
Soft LSTM:  0.0941 Topk LSTM:  0.0988


In [17]:
print("Benchmark Std: ")
print("LSTM: ", results_df_totalload["RMSE_LL_BiLSTM"].std().round(4), " , Transformer: ", results_df_totalload["RMSE_LL_Transformer"].std().round(4), " , CNN: ", results_df_totalload["RMSE_LL_CNN"].std().round(4))
print("MoE: ")
print("Soft Dense: ", results_df_totalload["RMSE_FL_soft_dense"].std().round(4), "Topk Dense: ", results_df_totalload["RMSE_FL_topk_dense"].std().round(4))
print("Soft LSTM: ", results_df_totalload["RMSE_FL_soft_bilstm"].std().round(4), "Topk LSTM: ", results_df_totalload["RMSE_FL_topk_bilstm"].std().round(4))

Benchmark Std: 
LSTM:  0.0228  , Transformer:  0.0256  , CNN:  0.0152
MoE: 
Soft Dense:  0.0232 Topk Dense:  0.0234
Soft LSTM:  0.0237 Topk LSTM:  0.0213


# PV

In [10]:
# Initialize DataFrame to store results
columns = ['User', 'Round']
model_names = ['BiLSTM', 'CNN', 'Transformer', 'soft_dense', 'soft_bilstm', 'topk_dense', 'topk_bilstm']

for name in model_names:
    columns.extend([f'RMSE_LL_{name}', f'MAE_LL_{name}', f'MSE_LL_{name}'])  # Local Learning Metrics
    columns.extend([f'RMSE_FL_{name}', f'MAE_FL_{name}', f'MSE_FL_{name}'])  # Federated Learning Metrics

results_df_pv = pd.DataFrame(columns=columns)

data = "PV"
user=1
cluster=4

#Load models
custom_objects = {'EinsumLayer': EinsumLayer,'TopKLayer': TopKLayer,'ImportanceRegularizationLayer': ImportanceRegularizationLayer}
local_model_paths = {
            'LL_BiLSTM': f'{cwd}/src/time_series/Locallearning/TS_LL_{data}_Forecasting_Benchmark/wandb/TS_LL_{data}_bilstm_u{user}_rd{1}.keras',
            'LL_CNN': f'{cwd}/src/time_series/Locallearning/TS_LL_{data}_Forecasting_Benchmark/wandb/TS_LL_{data}_cnn_u{user}_rd{1}.keras',
            'LL_Transformer': f'{cwd}/src/time_series/Locallearning/TS_LL_{data}_Forecasting_Benchmark/wandb/TS_LL_{data}_transformer_u{user}_rd{1}.keras',
            'LL_soft_bilstm': f'{cwd}/src/time_series/Locallearning/TS_LL_{data}_Forecasting_Moe/wandb/TS_LL_{data}_lstm_soft_moe_u{user}_rd{1}.keras',
            'LL_soft_dense': f'{cwd}/src/time_series/Locallearning/TS_LL_{data}_Forecasting_Moe/wandb/TS_LL_{data}_dense_soft_moe_u{user}_rd{1}.keras',
            'LL_topk_bilstm': f'{cwd}/src/time_series/Locallearning/TS_LL_{data}_Forecasting_Moe/wandb/TS_LL_{data}_lstm_topk_moe_u{user}_rd{1}.keras',
            'LL_topk_dense': f'{cwd}/src/time_series/Locallearning/TS_LL_{data}_Forecasting_Moe/wandb/TS_LL_{data}_dense_topk_moe_u{user}_rd{1}.keras'
        }

#Federated learning
federated_model_paths = {
            'FL_BiLSTM': f'{cwd}/src/time_series/Federatedlearning/TS_FL_{data}_Forecasting_Benchmark/wandb/TS_FL_{data}_global_bilstm_c{cluster}_FLround3.keras',
            'FL_CNN': f'{cwd}/src/time_series/Federatedlearning/TS_FL_{data}_Forecasting_Benchmark/wandb/TS_FL_{data}_global_cnn_c{cluster}_FLround3.keras',
            'FL_Transformer': f'{cwd}/src/time_series/Federatedlearning/TS_FL_{data}_Forecasting_Benchmark/wandb/TS_FL_{data}_global_transformer_c{cluster}_FLround3.keras',
            'FL_soft_bilstm': f'{cwd}/src/time_series/Federatedlearning/TS_FL_{data}_Forecasting_Moe/wandb/TS_FL_{data}_global_soft_bilstm_c{cluster}_FLround3.keras',
            'FL_soft_dense': f'{cwd}/src/time_series/Federatedlearning/TS_FL_{data}_Forecasting_Moe/wandb/TS_FL_{data}_global_soft_dense_c{cluster}_FLround3.keras',
            'FL_topk_bilstm': f'{cwd}/src/time_series/Federatedlearning/TS_FL_{data}_Forecasting_Moe/wandb/TS_FL_{data}_global_topk_bilstm_c{cluster}_FLround3.keras',
            'FL_topk_dense': f'{cwd}/src/time_series/Federatedlearning/TS_FL_{data}_Forecasting_Moe/wandb/TS_FL_{data}_global_topk_dense_c{cluster}_FLround3.keras'
        }

local_models = {name: load_and_compile_model(path, name, custom_objects) for name, path in local_model_paths.items()}
federated_models = {name: load_and_compile_model(path, name, custom_objects) for name, path in federated_model_paths.items()}

In [11]:
# Function to evaluate models
def evaluate_models(models, user_idx, X_test_user, y_test_user):
    user_metrics = {}
    for model_name, model in models.items():
        test_loss, test_rmse, test_mape, test_mae = model.evaluate(X_test_user, y_test_user, batch_size=batch_size, verbose=0)
        user_metrics[f'RMSE_{model_name}'] = test_rmse
        user_metrics[f'MAE_{model_name}'] = test_mae
        user_metrics[f'MSE_{model_name}'] = test_loss
    return user_metrics

# Iterate over each user
for user_idx in range(31, 60):
    
    # Get the test data for the current user
    X_test_user = X_test[f'user{user_idx}']
    y_test_user = y_test[f'user{user_idx}']
    
    # Evaluate local models
    local_metrics = evaluate_models(local_models, user_idx, X_test_user, y_test_user)
    
    # Evaluate federated models
    federated_metrics = evaluate_models(federated_models, user_idx, X_test_user, y_test_user)
    
    # Combine metrics and create a new row
    new_row = {'User': user_idx, 'Round': 1, **local_metrics, **federated_metrics}

    # Add the new row to the DataFrame
    results_df_pv.loc[len(results_df_pv)] = new_row

print(results_df_pv["RMSE_LL_BiLSTM"].mean())
print(results_df_pv["RMSE_FL_soft_bilstm"].mean())

0.1524841538790999
0.1125966798642586


In [18]:
print("Benchmark: ")
print("LSTM: ", results_df_pv["RMSE_LL_BiLSTM"].mean().round(4), " , Transformer: ", results_df_pv["RMSE_LL_Transformer"].mean().round(4), " , CNN: ", results_df_pv["RMSE_LL_CNN"].mean().round(4))
print("MoE: ")
print("Soft Dense: ", results_df_pv["RMSE_FL_soft_dense"].mean().round(4), "Topk Dense: ", results_df_pv["RMSE_FL_topk_dense"].mean().round(4))
print("Soft LSTM: ", results_df_pv["RMSE_FL_soft_bilstm"].mean().round(4), "Topk LSTM: ", results_df_pv["RMSE_FL_topk_bilstm"].mean().round(4))

Benchmark: 
LSTM:  0.1525  , Transformer:  0.149  , CNN:  0.1288
MoE: 
Soft Dense:  0.1208 Topk Dense:  0.1177
Soft LSTM:  0.1126 Topk LSTM:  0.1189


In [19]:
print("Benchmark Std: ")
print("LSTM: ", results_df_pv["RMSE_LL_BiLSTM"].std().round(4), " , Transformer: ", results_df_pv["RMSE_LL_Transformer"].std().round(4), " , CNN: ", results_df_pv["RMSE_LL_CNN"].std().round(4))
print("MoE: ")
print("Soft Dense: ", results_df_pv["RMSE_FL_soft_dense"].std().round(4), "Topk Dense: ", results_df_pv["RMSE_FL_topk_dense"].std().round(4))
print("Soft LSTM: ", results_df_pv["RMSE_FL_soft_bilstm"].std().round(4), "Topk LSTM: ", results_df_pv["RMSE_FL_topk_bilstm"].std().round(4))

Benchmark Std: 
LSTM:  0.0403  , Transformer:  0.0414  , CNN:  0.017
MoE: 
Soft Dense:  0.0308 Topk Dense:  0.0321
Soft LSTM:  0.0273 Topk LSTM:  0.0297


# Prosumption

In [13]:
# Initialize DataFrame to store results
columns = ['User', 'Round']
model_names = ['BiLSTM', 'CNN', 'Transformer', 'soft_dense', 'soft_bilstm', 'topk_dense', 'topk_bilstm']

for name in model_names:
    columns.extend([f'RMSE_LL_{name}', f'MAE_LL_{name}', f'MSE_LL_{name}'])  # Local Learning Metrics
    columns.extend([f'RMSE_FL_{name}', f'MAE_FL_{name}', f'MSE_FL_{name}'])  # Federated Learning Metrics

results_df_prosumption = pd.DataFrame(columns=columns)

data = "Prosumption"
user=1
cluster=4
#Load models
custom_objects = {'EinsumLayer': EinsumLayer,'TopKLayer': TopKLayer,'ImportanceRegularizationLayer': ImportanceRegularizationLayer}
local_model_paths = {
            'LL_BiLSTM': f'{cwd}/src/time_series/Locallearning/TS_LL_{data}_Forecasting_Benchmark/wandb/TS_LL_{data}_bilstm_u{user}_rd{1}.keras',
            'LL_CNN': f'{cwd}/src/time_series/Locallearning/TS_LL_{data}_Forecasting_Benchmark/wandb/TS_LL_{data}_cnn_u{user}_rd{1}.keras',
            'LL_Transformer': f'{cwd}/src/time_series/Locallearning/TS_LL_{data}_Forecasting_Benchmark/wandb/TS_LL_{data}_transformer_u{user}_rd{1}.keras',
            'LL_soft_bilstm': f'{cwd}/src/time_series/Locallearning/TS_LL_{data}_Forecasting_Moe/wandb/TS_LL_{data}_lstm_soft_moe_u{user}_rd{1}.keras',
            'LL_soft_dense': f'{cwd}/src/time_series/Locallearning/TS_LL_{data}_Forecasting_Moe/wandb/TS_LL_{data}_dense_soft_moe_u{user}_rd{1}.keras',
            'LL_topk_bilstm': f'{cwd}/src/time_series/Locallearning/TS_LL_{data}_Forecasting_Moe/wandb/TS_LL_{data}_lstm_topk_moe_u{user}_rd{1}.keras',
            'LL_topk_dense': f'{cwd}/src/time_series/Locallearning/TS_LL_{data}_Forecasting_Moe/wandb/TS_LL_{data}_dense_topk_moe_u{user}_rd{1}.keras'
        }

#Federated learning
federated_model_paths = {
            'FL_BiLSTM': f'{cwd}/src/time_series/Federatedlearning/TS_FL_{data}_Forecasting_Benchmark/wandb/TS_FL_{data}_global_bilstm_c{cluster}_FLround3.keras',
            'FL_CNN': f'{cwd}/src/time_series/Federatedlearning/TS_FL_{data}_Forecasting_Benchmark/wandb/TS_FL_{data}_global_cnn_c{cluster}_FLround3.keras',
            'FL_Transformer': f'{cwd}/src/time_series/Federatedlearning/TS_FL_{data}_Forecasting_Benchmark/wandb/TS_FL_{data}_global_transformer_c{cluster}_FLround3.keras',
            'FL_soft_bilstm': f'{cwd}/src/time_series/Federatedlearning/TS_FL_{data}_Forecasting_Moe/wandb/TS_FL_{data}_global_soft_bilstm_c{cluster}_FLround3.keras',
            'FL_soft_dense': f'{cwd}/src/time_series/Federatedlearning/TS_FL_{data}_Forecasting_Moe/wandb/TS_FL_{data}_global_soft_dense_c{cluster}_FLround3.keras',
            'FL_topk_bilstm': f'{cwd}/src/time_series/Federatedlearning/TS_FL_{data}_Forecasting_Moe/wandb/TS_FL_{data}_global_topk_bilstm_c{cluster}_FLround3.keras',
            'FL_topk_dense': f'{cwd}/src/time_series/Federatedlearning/TS_FL_{data}_Forecasting_Moe/wandb/TS_FL_{data}_global_topk_dense_c{cluster}_FLround3.keras'
        }

local_models = {name: load_and_compile_model(path, name, custom_objects) for name, path in local_model_paths.items()}
federated_models = {name: load_and_compile_model(path, name, custom_objects) for name, path in federated_model_paths.items()}

In [14]:
# Function to evaluate models
def evaluate_models(models, user_idx, X_test_user, y_test_user):
    user_metrics = {}
    for model_name, model in models.items():
        test_loss, test_rmse, test_mape, test_mae = model.evaluate(X_test_user, y_test_user, batch_size=batch_size, verbose=0)
        user_metrics[f'RMSE_{model_name}'] = test_rmse
        user_metrics[f'MAE_{model_name}'] = test_mae
        user_metrics[f'MSE_{model_name}'] = test_loss
    return user_metrics

# Iterate over each user
for user_idx in range(31, 60):
    
    # Get the test data for the current user
    X_test_user = X_test[f'user{user_idx}']
    y_test_user = y_test[f'user{user_idx}']
    
    # Evaluate local models
    local_metrics = evaluate_models(local_models, user_idx, X_test_user, y_test_user)
    
    # Evaluate federated models
    federated_metrics = evaluate_models(federated_models, user_idx, X_test_user, y_test_user)
    
    # Combine metrics and create a new row
    new_row = {'User': user_idx, 'Round': 1, **local_metrics, **federated_metrics}

    # Add the new row to the DataFrame
    results_df_prosumption.loc[len(results_df_prosumption)] = new_row

print(results_df_prosumption["RMSE_LL_BiLSTM"].mean())
print(results_df_prosumption["RMSE_FL_soft_bilstm"].mean())

0.21469462020643826
0.10607381113644304


In [20]:
print("Benchmark: ")
print("LSTM: ", results_df_prosumption["RMSE_LL_BiLSTM"].mean().round(4), " , Transformer: ", results_df_prosumption["RMSE_LL_Transformer"].mean().round(4), " , CNN: ", results_df_prosumption["RMSE_LL_CNN"].mean().round(4))
print("MoE: ")
print("Soft Dense: ", results_df_prosumption["RMSE_FL_soft_dense"].mean().round(4), "Topk Dense: ", results_df_prosumption["RMSE_FL_topk_dense"].mean().round(4))
print("Soft LSTM: ", results_df_prosumption["RMSE_FL_soft_bilstm"].mean().round(4), "Topk LSTM: ", results_df_prosumption["RMSE_FL_topk_bilstm"].mean().round(4))

Benchmark: 
LSTM:  0.2147  , Transformer:  0.1819  , CNN:  0.3384
MoE: 
Soft Dense:  0.1141 Topk Dense:  0.115
Soft LSTM:  0.1061 Topk LSTM:  0.1147


In [21]:
print("Benchmark Std: ")
print("LSTM: ", results_df_prosumption["RMSE_LL_BiLSTM"].std().round(4), " , Transformer: ", results_df_prosumption["RMSE_LL_Transformer"].std().round(4), " , CNN: ", results_df_prosumption["RMSE_LL_CNN"].std().round(4))
print("MoE: ")
print("Soft Dense: ", results_df_prosumption["RMSE_FL_soft_dense"].std().round(4), "Topk Dense: ", results_df_prosumption["RMSE_FL_topk_dense"].std().round(4))
print("Soft LSTM: ", results_df_prosumption["RMSE_FL_soft_bilstm"].std().round(4), "Topk LSTM: ", results_df_prosumption["RMSE_FL_topk_bilstm"].std().round(4))

Benchmark Std: 
LSTM:  0.0137  , Transformer:  0.0327  , CNN:  0.0328
MoE: 
Soft Dense:  0.0326 Topk Dense:  0.0327
Soft LSTM:  0.0345 Topk LSTM:  0.0322
